In [3]:
from IPython.display import clear_output
import numpy as np

Rotation stuffs

In [4]:
vecs = np.array(list((np.random.random(DIM * DATA_COUNT) * 2 * DATA_LIMIT) - DATA_LIMIT))
vecs = np.resize(vecs, (DATA_COUNT, DIM))
vecs = np.insert(vecs, DIM, np.zeros(DATA_COUNT), axis=1)

def rotate_vec(vec, theta=None, i=None, j=None):

    dim = vec.size
    if theta is None:
        theta = 2 * np.pi * np.random.random()

    if i is None:
        i = np.random.choice(dim)
    if j is None:
        j = np.random.choice(dim)
        while i == j:
            j = np.random.choice(dim)
    rotational_matrix = np.identity(dim)
    rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
    rotational_matrix[i][j] = np.sin(theta)
    rotational_matrix[j][i] = -np.sin(theta)

    return np.matmul(rotational_matrix, vec), rotational_matrix


tot_rot = np.identity(DIM + 1)
for i in range(21):
    _, rotvect = rotate_vec(np.zeros(DIM + 1))
    tot_rot = np.matmul(rotvect, tot_rot)
    vecs = np.array([np.matmul(rotvect, vec) for vec in vecs])


Creating data

In [5]:
DATA_COUNT = 100
DIM = 1
DATA_LIMIT = 10

def tensor_diff(a, b):
    c = a - b
    c = np.squeeze(c)
    c_t = np.var(c)
    c = np.square(c)
    c = np.sqrt(np.average(c))
    return np.average(c), c_t

def dropout_p_matrix(percent,shp, step = True):
    dropout_p = np.random.random(shp)
    d_l = np.vectorize(lambda x : 1 if x < percent else 0)
    if step:
        return(d_l(dropout_p))
    else:
        return dropout_p

def hyp(vex):
    np.random.shuffle(vex.T)
    return vex

Making the auto-encoder

In [6]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [7]:

def regressutil(weights, training_data, training_label, epochs, testing_data, testing_label,alpha = 0.001, update_interval = 10, retain = 1):
    for epoch in range(epochs):
        predicted_label = training_data.dot(weights * dropout_p_matrix(retain, weights.shape))
        
        dW = - (2 * training_data.T.dot(training_label - predicted_label)) / training_data.size
        weights -= dW * alpha

        if epoch % (epochs // update_interval) == 0:
            clear_output()
            print("Error :", tensor_diff(testing_data.dot(weights), testing_label))
            print("Weight")
            print(weights)
    return weights

        

In [8]:
def regress(W, vecs, vecsy, epochs):
    vecs = np.array(np.split(vecs, 10))
    vecsy = np.array(np.split(vecsy, 10))
    for i in range(10):
        W = regressutil(W, np.vstack(np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0)), epochs //10, vecs[i], vecsy[i], update_interval = 1, retain = (i/30 + 0.7))

    return W

In [15]:
W = np.random.random((DIM + 1, DIM + 1))
vecs = (np.random.random((DATA_COUNT, (DIM + 1))) * 2 * DATA_LIMIT) - DATA_LIMIT
vecsy = hyp(vecs.copy())

In [10]:
regress(W, vecs, vecsy, 800000)

Error : (0.08287495428286819, 0.0067370315037364615)
Weight
[[-0.0001  1.0178]
 [ 1.0081 -0.    ]]


array([[0., 1.],
       [1., 0.]])

In [19]:
regressutil(W, vecs, vecsy, 80000, vecs, vecsy, update_interval = 10, retain = 1, alpha= 0.001)

Error : (1.8478597904844023e-14, 3.413161449083801e-28)
Weight
[[1. 0.]
 [0. 1.]]


array([[1., 0.],
       [0., 1.]])